In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer
import pandas as pd
from sklearn.metrics import f1_score, classification_report
import os

# 커스텀 모델 로드
from model_training_2 import MultiLabelClassifier

# 모델과 토크나이저 로드
model_path = r"C:\Users\user\OneDrive\바탕 화면\project\OSS_Project\AI\text-model\models\bert_model"
checkpoint = torch.load(os.path.join(model_path, "model.pt"))
model = MultiLabelClassifier()
model.load_state_dict(checkpoint['model_state_dict'])
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 테스트 데이터 로드
data_path = r"C:\Users\user\OneDrive\바탕 화면\project\OSS_Project\AI\text-model\data\processed_data.csv"
data = pd.read_csv(data_path)

# 입력 문장 토크나이징
inputs = tokenizer(list(data['입력 문장']), padding=True, truncation=True, return_tensors="pt")

# 레이블 매핑
도수_매핑 = {'낮은': 0, '중간': 1, '높은': 2}
술종류_매핑 = {'칵테일': 0, '럼': 1, '위스키': 2, '보드카': 3}
맛_매핑 = {'달달한': 0, '쓴맛': 1, '상큼한': 2, '신맛': 3, '부드러운': 4}

# 레이블 변환
도수_labels = torch.tensor([도수_매핑[도수] for 도수 in data['도수']])
술종류_labels = torch.tensor([술종류_매핑[종류] for 종류 in data['술 종류']])
맛_labels = torch.tensor([맛_매핑[맛] for 맛 in data['맛']])

# 데이터셋 생성
dataset = TensorDataset(
    inputs['input_ids'], 
    inputs['attention_mask'], 
    도수_labels,
    술종류_labels,
    맛_labels
)
dataloader = DataLoader(dataset, batch_size=2)

# 모델 평가
model.eval()
all_preds_도수 = []
all_preds_술종류 = []
all_preds_맛 = []
all_labels_도수 = []
all_labels_술종류 = []
all_labels_맛 = []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, 도수_label, 술종류_label, 맛_label = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # 각 분류기의 예측값 계산
        도수_pred = torch.argmax(outputs['도수'], dim=1)
        술종류_pred = torch.argmax(outputs['술종류'], dim=1)
        맛_pred = torch.argmax(outputs['맛'], dim=1)
        
        # 예측값과 실제값 저장
        all_preds_도수.extend(도수_pred.cpu().numpy())
        all_preds_술종류.extend(술종류_pred.cpu().numpy())
        all_preds_맛.extend(맛_pred.cpu().numpy())
        all_labels_도수.extend(도수_label.cpu().numpy())
        all_labels_술종류.extend(술종류_label.cpu().numpy())
        all_labels_맛.extend(맛_label.cpu().numpy())

# 각 분류기별 성능 평가
print("도수 분류 결과:")
print(classification_report(all_labels_도수, all_preds_도수, 
                          target_names=['낮은', '중간', '높은']))

print("\n술종류 분류 결과:")
print(classification_report(all_labels_술종류, all_preds_술종류, 
                          target_names=['칵테일', '럼', '위스키', '보드카']))

print("\n맛 분류 결과:")
print(classification_report(all_labels_맛, all_preds_맛, 
                          target_names=['달달한', '쓴맛', '상큼한', '신맛', '부드러운']))

ModuleNotFoundError: No module named 'model_training_2'